In [1]:
!pip install openai

In [2]:
import getpass
import os

# set env for openai key:

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [3]:
from openai import OpenAI
import pprint
client = OpenAI()

In [4]:
from enum import Enum
from typing import List

from pydantic import BaseModel


class UIType(str, Enum):
    div = "div"
    button = "button"
    header = "header"
    section = "section"
    field = "field"
    form = "form"

class Attribute(BaseModel):
    name: str
    value: str

class UI(BaseModel):
    type: UIType
    label: str
    children: List["UI"]
    attributes: List[Attribute]

UI.model_rebuild()  # This is required to enable recursive types

class Response(BaseModel):
    ui: UI

response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {
            "role": "system",
            "content": "You are a UI generator AI. Convert the user input into a UI.",
        },
        {"role": "user", "content": "Make a User Profile Form"},
    ],
    text_format=Response,
)

ui = response.output_parsed

# chain? o seguimos la conversacion?

chainUI = client.responses.create(
    model="gpt-4o",
    previous_response_id=response.id, # esta es la forma de persistir el puntero de la llamada anterior
    input=[{"role": "user", "content": "convierte el UI en JSX y devuelve solo el codigo."}],
)
print(chainUI.output_text)


```jsx
import React from 'react';

function UserProfileForm() {
  return (
    <form>
      <h1>User Profile</h1>
      <label>
        First Name
        <input type="text" placeholder="Enter your first name" />
      </label>
      <br />
      <label>
        Last Name
        <input type="text" placeholder="Enter your last name" />
      </label>
      <br />
      <label>
        Email
        <input type="email" placeholder="Enter your email" />
      </label>
      <br />
      <label>
        Phone Number
        <input type="tel" placeholder="Enter your phone number" />
      </label>
      <br />
      <label>
        Address
        <input type="text" placeholder="Enter your address" />
      </label>
      <br />
      <label>
        Profile Picture
        <input type="file" />
      </label>
      <br />
      <button type="submit">Submit</button>
    </form>
  );
}

export default UserProfileForm;
```


# LANGCHAIN

In [5]:
# https://python.langchain.com/docs/concepts/prompt_templates/
# install langchain

!pip install langchain
!pip install -qU "langchain[openai]"

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

# Nivel 1 con langchain

# define model to use
model = init_chat_model("gpt-4o-mini", model_provider="openai")

# message preparation:

## using history messages:
messages = [
    SystemMessage("Translate the following from English into Spanish"),
    HumanMessage("hi! Im a developer trying to learn AI"),
]

# LLM call

## invoke chat completion
response = model.invoke(messages)

# print response text
print("response.content:\n")
pprint.pprint(response.content + "\n\n")
print("response object:\n")
pprint.pprint(response.model_dump())


response.content:

'¡Hola! Soy un desarrollador que está tratando de aprender sobre IA.\n'
response object:

{'additional_kwargs': {'refusal': None},
 'content': '¡Hola! Soy un desarrollador que está tratando de aprender sobre '
            'IA.',
 'example': False,
 'id': 'run-a257c36c-6c22-4f0d-b6e6-4603c9c26fc3-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'id': 'chatcmpl-BPe7Ez0ryeo4arF36pBrhAfC7Qx8L',
                       'logprobs': None,
                       'model_name': 'gpt-4o-mini-2024-07-18',
                       'system_fingerprint': 'fp_0392822090',
                       'token_usage': {'completion_tokens': 16,
                                       'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                     'audio_tokens': 0,
                                                                     'reasoning_tokens': 0,
       

In [8]:
# GENERA EL TEXTO USANDO UN PROMPT CON VARIABLES EN LANGCHAIN
# Nivel 4 con langchain

## using template with variables
system_template = "Translate the following into {language}" # variable on template interpolated with {var_name}

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template), 
        ("user", "{text}")
    ]
)

template_run = prompt_template | model

prompt_template_response = template_run.invoke(
    {
        "language": "Spanish", 
        "text": "hi! Please take the recommended actions to update your configuration"
    })

pprint.pprint(prompt_template_response.content)



('¡Hola! Por favor, toma las acciones recomendadas para actualizar tu '
 'configuración.')


In [9]:
template_prompt = ChatPromptTemplate.from_template("Dime un efemeride del siguiente tema: {topic}")

chain = template_prompt | model
response1 = chain.invoke({"topic": "Futbol"})
pprint.pprint(response1.content)

('Una efeméride importante en el mundo del fútbol es el 22 de junio de 1986, '
 'fecha en la cual Diego Maradona anotó el famoso "Gol de la Mano de Dios" y '
 'el "Gol del Siglo" durante el partido de cuartos de final de la Copa del '
 'Mundo de la FIFA en México, donde Argentina se enfrentó a Inglaterra. Estos '
 'dos goles se consideran algunos de los momentos más icónicos en la historia '
 'del fútbol. Argentina ganó ese partido 2-1 y posteriormente se consagró '
 'campeón del torneo.')
